In [ ]:
# Setup
import os
import json
from dotenv import load_dotenv
from llama_stack_client import LlamaStackClient

# Load configuration
load_dotenv('config.env')
base_url = os.environ.get('LLAMA_STACK_URL', 'http://localhost:8321')
model_id = os.environ.get('LLM_MODEL_ID', 'r1-qwen-14b-w4a16')

print(f"🔗 Connecting to: {base_url}")
print(f"🤖 Using model: {model_id}")

# Create client
client = LlamaStackClient(base_url=base_url)
print("✅ Client created")

# Check available MCP tools
print("\n🔍 Checking available MCP Atlassian tools...")
try:
    tools = client.tools.list(toolgroup_id="mcp::atlassian")
    print(f"✅ Found {len(tools)} Atlassian tools")
    
    # Find Jira tools
    jira_tools = [t for t in tools if 'jira' in str(t).lower()]
    print(f"🎫 Jira tools: {len(jira_tools)}")
    
    # Show create and search tools
    for tool in jira_tools:
        tool_name = getattr(tool, 'name', getattr(tool, 'identifier', str(tool)))
        if 'create' in tool_name or 'search' in tool_name:
            print(f"  - {tool_name}")
            
except Exception as e:
    print(f"❌ Error loading tools: {e}")


In [ ]:
# Hybrid Function: Conversations API + MCP Tools
def create_jira_with_conversations(prompt, previous_response_id=None):
    """Use Conversations API for conversation flow and MCP tools for Jira operations"""
    print(f"\n🧪 Creating Jira issue with Conversations API: {prompt}")
    print("="*70)
    
    try:
        # Step 1: Use Conversations API for natural conversation
        print("🤖 Step 1: Using Conversations API for conversation flow...")
        
        # First, get the AI's response about what to do
        conversation_response = client.responses.create(
            model=model_id,
            input=f"""You are a Jira assistant. The user wants to: {prompt}
            
            Please analyze this request and provide:
            1. A clear summary of what Jira issue needs to be created
            2. The specific parameters needed (project_key, summary, issue_type, description, priority, labels)
            3. A confirmation that you understand the request
            
            Format your response as JSON with these fields:
            - summary: Brief description of the issue
            - project_key: The Jira project key
            - issue_type: Type of issue (Task, Bug, Incident, etc.)
            - description: Detailed description
            - priority: Priority level (Low, Medium, High, Critical)
            - labels: List of labels to apply
            - confirmation: Your understanding of the request""",
            previous_response_id=previous_response_id
        )
        
        print(f"✅ Conversations API response received!")
        print(f"Response ID: {conversation_response.id}")
        print(f"Response content: {conversation_response.output[0].content[0].text}")
        
        # Step 2: Parse the AI's response and extract parameters
        print("\n🔧 Step 2: Parsing AI response and extracting parameters...")
        
        # Try to extract JSON from the response
        try:
            # Look for JSON in the response content
            content = conversation_response.output[0].content[0].text
            if '{' in content and '}' in content:
                # Extract JSON part
                start = content.find('{')
                end = content.rfind('}') + 1
                json_str = content[start:end]
                issue_params = json.loads(json_str)
                print(f"✅ Extracted parameters: {issue_params}")
            else:
                # Fallback: create basic parameters from the prompt
                issue_params = {
                    "summary": "Conversations API Generated Issue",
                    "project_key": "KAN",
                    "issue_type": "Task",
                    "description": prompt,
                    "priority": "Medium",
                    "labels": ["conversations-api", "auto-generated"]
                }
                print(f"⚠️  Using fallback parameters: {issue_params}")
        except Exception as e:
            print(f"⚠️  JSON parsing failed, using fallback: {e}")
            issue_params = {
                "summary": "Conversations API Generated Issue",
                "project_key": "KAN",
                "issue_type": "Task",
                "description": prompt,
                "priority": "Medium",
                "labels": ["conversations-api", "auto-generated"]
            }
        
        # Step 3: Execute MCP tool with extracted parameters
        print("\n🛠️  Step 3: Executing MCP tool with extracted parameters...")
        
        # Prepare additional_fields
        additional_fields = {}
        if "priority" in issue_params:
            additional_fields["priority"] = {"name": issue_params["priority"]}
        if "labels" in issue_params:
            additional_fields["labels"] = issue_params["labels"]
        
        # Call MCP tool
        mcp_result = client.tool_runtime.invoke_tool(
            tool_name="jira_create_issue",
            kwargs={
                "project_key": issue_params.get("project_key", "KAN"),
                "summary": issue_params.get("summary", "Conversations API Issue"),
                "issue_type": issue_params.get("issue_type", "Task"),
                "description": issue_params.get("description", prompt),
                "additional_fields": additional_fields
            }
        )
        
        print("✅ MCP tool executed successfully!")
        print(f"Result: {mcp_result}")
        
        # Step 4: Get final confirmation from Conversations API
        print("\n🤖 Step 4: Getting final confirmation from Conversations API...")
        
        final_response = client.responses.create(
            model=model_id,
            input=f"""The Jira issue has been created successfully! Here are the details:
            
            Issue Parameters Used:
            - Project: {issue_params.get('project_key', 'KAN')}
            - Summary: {issue_params.get('summary', 'Conversations API Issue')}
            - Type: {issue_params.get('issue_type', 'Task')}
            - Description: {issue_params.get('description', prompt)}
            - Priority: {issue_params.get('priority', 'Medium')}
            - Labels: {issue_params.get('labels', [])}
            
            MCP Tool Result: {mcp_result}
            
            Please provide a friendly confirmation message to the user that the issue was created successfully.""",
            previous_response_id=conversation_response.id
        )
        
        print(f"✅ Final confirmation received!")
        print(f"Final response: {final_response.output[0].content[0].text}")
        
        return {
            "conversation_response": conversation_response,
            "issue_params": issue_params,
            "mcp_result": mcp_result,
            "final_response": final_response
        }
        
    except Exception as e:
        print(f"❌ Error: {e}")
        import traceback
        traceback.print_exc()
        return None


In [ ]:
# Test 1: Simple Jira Issue Creation using Hybrid Approach
print("🧪 Test 1: Create Simple Jira Issue using Hybrid Approach")
print("="*60)

result1 = create_jira_with_conversations(
    "Create a Jira issue in the KAN project with summary 'Hybrid Conversations API Test' and type 'Task'"
)

if result1:
    print("🎉 Test 1 SUCCESS: Issue created using hybrid approach!")
    print(f"Conversation Response ID: {result1['conversation_response'].id}")
    print(f"Final Response ID: {result1['final_response'].id}")
else:
    print("❌ Test 1 FAILED: Issue not created")


In [ ]:
# Test 2: OOM Error Incident Creation using Hybrid Approach
print("\n🧪 Test 2: Create OOM Error Incident using Hybrid Approach")
print("="*60)

result2 = create_jira_with_conversations(
    """Create a Jira incident for a pod failing due to OOM error in the KAN project:
    - Summary: 'Pod failing due to OOM error'
    - Issue Type: 'Incident'
    - Description: 'Pod experiencing Out of Memory errors causing application failures'
    - Priority: High
    - Labels: ['oom-error', 'pod-failure', 'high-priority']""",
    previous_response_id=result1['conversation_response'].id if result1 else None
)

if result2:
    print("🎉 Test 2 SUCCESS: OOM incident created using hybrid approach!")
    print(f"Conversation Response ID: {result2['conversation_response'].id}")
    print(f"Final Response ID: {result2['final_response'].id}")
else:
    print("❌ Test 2 FAILED: OOM incident not created")


In [ ]:
# Test 3: Verify Issues Were Created
print("\n🧪 Test 3: Verify Issues Were Created")
print("="*40)

try:
    # Use direct tool call to verify issues were created
    search_result = client.tool_runtime.invoke_tool(
        tool_name="jira_search",
        kwargs={"jql": "project = KAN ORDER BY created DESC"}
    )
    
    print("✅ Direct search completed!")
    
    # Check if our test issues were created
    search_text = str(search_result)
    if "Hybrid Conversations API Test" in search_text:
        print("🎉 SUCCESS: Hybrid Conversations API Test issue found!")
    else:
        print("⚠️  Hybrid Conversations API Test issue not found")
        
    if "OOM error" in search_text:
        print("🎉 SUCCESS: OOM error incident found!")
    else:
        print("⚠️  OOM error incident not found")
        
    print(f"\nRecent issues in KAN project:")
    print(search_result)
        
except Exception as e:
    print(f"❌ Search failed: {e}")
